In [310]:
# Import requests to retrive Web Urls example HTML. TXT 
import requests

# Import BeautifulSoup
from bs4 import BeautifulSoup

# import re module for REGEXes
import re

# import pandas
import pandas as pd
from sec_edgar_downloader import Downloader
import utils
import os
newPath = downloadPath + "sec_edgar_filings"
downloadPath = "./"
dl = Downloader(downloadPath)

In [517]:
# ticker = "ADBE"
# ticker = 'TSLA' ## "0001564590-15-001031.txt" Exception case

# ticker = "GE" ## Exception https://www.sec.gov/Archives/edgar/data/40545/000004054515000030/ge10k2014.htm

# ticker = "COF" ## done
# ticker = "LHX" ## done
# ticker = "ICE" # regex change done
# ticker = "DE" # regex change
# ticker = "AON" # https://www.sec.gov/Archives/edgar/data/315293/000162828017001652/0001628280-17-001652-index.htm 
# this onwards otherwise, table of the contents at the end 
# ticker = "NSC"
# ticker = "NOC" Exception for year 2019, 2020 
# ticker = "GPN" # done
# ticker = "VRTX" # done
# tiker = "CL" # exception handle for cases in year 2011, 2010 filings iteam and 1a in different tags
# ticker = "MS" # exception for filings after and including 2018, no item 1a format but Risk Factors format
# ticker = "PNC" # exception for years below 2015, working for 2015, iteam 1 a in seperate tags
# ticker = "TFC" # Look for item 2 if item b start is smaller than the iteam a start
# ticker = "CAT"
# ticker = "RJF"
# ticker = "HST" # item 4 after item 1a
# ticker = "MKTX" #item 2 after item 1a
# ticker = "VAR" # exception only for year 2013 item 1a in different tags
# ticker = "EXAS"
# ticker = "IT" # exception for year 2012, item 1a different tags
# ticker = "INCY" 
# ticker = "TTWO"
# ticker = "HP"
# ticker = "AR"
# ticker = "USM" # Exception can't extract for year 2016 and onwards
ticker = "IAA"

In [518]:
utils.downloadFilings(dl, ticker)
companyFilingsPath = os.path.join(newPath, ticker, "10-K")
companyFilings = [(os.path.join(companyFilingsPath, f), f) for f in os.listdir(companyFilingsPath)]
filingPath = companyFilings[0][0]
# f = open(filingPath,'r')
# f = open(os.path.join(companyFilingsPath, "0000821130-16-000050.txt"))
# raw_10k = f.read()
# f.close()

In [ ]:
print(raw_10k[0:1300])

In [513]:
# Regex to find <DOCUMENT> tags
doc_start_pattern = re.compile(r'<DOCUMENT>')
doc_end_pattern = re.compile(r'</DOCUMENT>')
# Regex to find <TYPE> tag prceeding any characters, terminating at new line
type_pattern = re.compile(r'<TYPE>[^\n]+')

In [514]:
# Create 3 lists with the span idices for each regex

### There are many <Document> Tags in this text file, each as specific exhibit like 10-K, EX-10.17 etc
### First filter will give us document tag start <end> and document tag end's <start> 
### We will use this to later grab content in between these tags
doc_start_is = [x.end() for x in doc_start_pattern.finditer(raw_10k)]
doc_end_is = [x.start() for x in doc_end_pattern.finditer(raw_10k)]

### Type filter is interesting, it looks for <TYPE> with Not flag as new line, ie terminare there, with + sign
### to look for any char afterwards until new line \n. This will give us <TYPE> followed Section Name like '10-K'
### Once we have have this, it returns String Array, below line will with find content after <TYPE> ie, '10-K' 
### as section names
doc_types = [x[len('<TYPE>'):] for x in type_pattern.findall(raw_10k)]
# doc_types

In [515]:
document = {}

# Create a loop to go through each section type and save only the 10-K section in the dictionary
for doc_type, doc_start, doc_end in zip(doc_types, doc_start_is, doc_end_is):
    if doc_type == '10-K'and doc_type not in document.keys():
        document[doc_type] = raw_10k[doc_start:doc_end]
print(document.keys())

dict_keys(['10-K'])


In [184]:
document['10-K'][0:500]

'\n<TYPE>10-K\n<SEQUENCE>1\n<FILENAME>a2233505z10-k.htm\n<DESCRIPTION>10-K\n<TEXT>\n<HTML>\n<HEAD>\n</HEAD>\n<BODY BGCOLOR="#FFFFFF" LINK=BLUE  VLINK=PURPLE>\n<BR>\n<P><FONT SIZE=3 >\nUse these links to rapidly review the document<BR>\n<A HREF="#bg14602_table_of_contents">  TABLE OF CONTENTS</A> <BR>\n<A HREF="#ci14602_item_15._exhibits_and_financial_statement_schedules.">  ITEM 15. EXHIBITS AND FINANCIAL STATEMENT SCHEDULES.</A><BR></font>\n</P>\n\n<P style="font-family:times;"><FONT SIZE=2>\n\n<!-- COMMAND=ADD_BA'

In [ ]:
document['10-K']

In [516]:
# >ITEM&#160;1&#160;(B).&#160;&#160;&#160;&#160;
# Write the regex
regex = re.compile(r'(>(\s|&#160;|&nbsp;)*item(\s|&#160;|&nbsp;)*(1(\s|&nbsp;|&#160;|&#160;\(|\(|)*a|6|1(\s|&nbsp;|&#160;|&#160;\(|\(|)*b|7(\s|&nbsp;|&#160;|&#160;\(|\(|)*a|7|8|2|4)\.{0,1})', re.I)

# Use finditer to math the regex
matches = regex.finditer(document['10-K'])

# Write a for loop to print the matches
for match in matches:
    print(match)

<re.Match object; span=(137037, 137046), match='>Item 1A.'>
<re.Match object; span=(138047, 138056), match='>Item 1B.'>
<re.Match object; span=(139070, 139078), match='>Item 2.'>
<re.Match object; span=(141077, 141085), match='>Item 4.'>
<re.Match object; span=(147196, 147204), match='>Item 6.'>
<re.Match object; span=(148210, 148218), match='>Item 7.'>
<re.Match object; span=(149287, 149296), match='>Item 7A.'>
<re.Match object; span=(150345, 150353), match='>Item 8.'>


In [504]:
# Matches
matches = regex.finditer(document['10-K'])

# Create the dataframe
test_df = pd.DataFrame([(x.group(), x.start(), x.end()) for x in matches])

test_df.columns = ['item', 'start', 'end']
test_df['item'] = test_df.item.str.lower()

# Display the dataframe
test_df

,item,start,end
0,>item&nbsp;1a.,19031,19045
1,>item&nbsp;1b.,19898,19912
2,>item&nbsp;4.,21671,21684
3,>item&nbsp;6.,24130,24143
4,>item&nbsp;7.,25033,25046
5,>item&nbsp;7a.,26016,26030
6,>item&nbsp;8.,26984,26997
7,> item&nbsp;1a.,483779,483795
8,> item&nbsp;1b.,589463,589479
9,> item&nbsp;4.,591507,591522


In [505]:
# Get rid of unnesesary charcters from the dataframe
test_df.replace('&#160;ris',' ',regex=True,inplace=True)
test_df.replace('&#160;unresolve',' ',regex=True,inplace=True)
test_df.replace('&#160;',' ',regex=True,inplace=True)
test_df.replace('&nbsp;',' ',regex=True,inplace=True)
test_df.replace(' ','',regex=True,inplace=True)
test_df.replace('\.','',regex=True,inplace=True)
test_df.replace('>','',regex=True,inplace=True)
test_df.replace('\n','',regex=True,inplace=True)
test_df.replace('\(','',regex=True,inplace=True)

# display the dataframe
test_df

,item,start,end
0,item1a,19031,19045
1,item1b,19898,19912
2,item4,21671,21684
3,item6,24130,24143
4,item7,25033,25046
5,item7a,26016,26030
6,item8,26984,26997
7,item1a,483779,483795
8,item1b,589463,589479
9,item4,591507,591522


In [506]:
def customsort(x):
    if len(x)>1:
        x = x.values
        x.sort()
        return x[1]
    else:
        return x[0]
def customsort2(x):
    x = x.values
    x.sort()
    return x[0]
#     print("left")

In [507]:
pos_dat = test_df.groupby(['item']).agg({'start': customsort, 'end': 'max'})
# pos_dat.set_index('item', inplace=True)
list(pos_dat.index)
pos_dat

,start,end
item,,
item1a,483779,483795
item1b,589463,589479
item4,591507,591522
item6,603912,603927
item7,820202,820217
item7a,1182940,1182956
item8,1194975,1194990


In [508]:
# Get Item 1a
item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['end'].loc['item1b']]
# item_1a_raw = document['10-K'][pos_dat['start'].loc['item1a']:pos_dat['end'].loc['item2']]

# Get Item 7
# item_7_raw = document['10-K'][pos_dat['start'].loc['item7']:pos_dat['start'].loc['item7a']]

# item_6_raw = document['10-K'][pos_dat['start'].loc['item6']:pos_dat['start'].loc['item7']]

# Get Item 7a
# item_7a_raw = document['10-K'][pos_dat['start'].loc['item7a']:pos_dat['start'].loc['item8']]

In [86]:
item_1a_raw

'>Item 1A.</div>\n</td>\n<td style="BORDER-BOTTOM: #000000 0.5pt dotted; WIDTH: 2.62%; FONT-FAMILY: Arial; VERTICAL-ALIGN: bottom">&#160;</td>\n<td style="BORDER-BOTTOM: #000000 0.5pt dotted; WIDTH: 44.87%; VERTICAL-ALIGN: bottom">\n<div style="TEXT-ALIGN: left; LINE-HEIGHT: 12pt; FONT-FAMILY: Arial; COLOR: #000000; FONT-SIZE: 8pt">Risk Factors</div>\n</td>\n<td style="BORDER-BOTTOM: #000000 0.5pt dotted; WIDTH: 20%; FONT-FAMILY: Arial; VERTICAL-ALIGN: bottom">&#160;</td>\n<td style="BORDER-BOTTOM: #000000 0.5pt dotted; WIDTH: 20%; VERTICAL-ALIGN: bottom">\n<div style="TEXT-ALIGN: center; LINE-HEIGHT: 12pt; FONT-FAMILY: Arial; COLOR: #000000; FONT-SIZE: 8pt">112-117</div>\n</td>\n</tr>\n\n<tr>\n<td style="WIDTH: 12.66%; VERTICAL-ALIGN: bottom; BORDER-TOP: #000000 0.5pt dotted">\n<div style="TEXT-ALIGN: left; LINE-HEIGHT: 9.1pt; FONT-FAMILY: \'Times New Roman\'; FONT-SIZE: 8pt">&#160;</div>\n\n<div style="TEXT-ALIGN: left; LINE-HEIGHT: 9.1pt; FONT-FAMILY: Arial; FONT-SIZE: 8pt">Item 1B.

In [ ]:
item_6_raw[0:1000]

In [509]:
### First convert the raw text we have to exrtacted to BeautifulSoup object 
item_1a_content = BeautifulSoup(item_1a_raw, "lxml")
# item_6_content = BeautifulSoup(item_6_raw, 'lxml')
# item_7_content = BeautifulSoup(item_7_raw, 'lxml')

In [410]:
# print(item_1a_content.get_text())

>Item 1A.Risk FactorsPages 46-53Item 1B.


In [510]:
import unicodedata
text = item_1a_content.get_text()
# regex = re.compile(r'item\s(1a|6|1B|7a|7|8)\.{0,1}', re.I)
text = unicodedata.normalize("NFKD", text)
text = text.replace("\n", " ")
text = text.replace("  ", " ")
text = text.lower()
print(text)
# first = text.find("item 1a")
# # print(first)
# second = text.find("item 1a.", first+1)
# print(text[second:])
# matches = regex.finditer(text)
# print(text)
# text
# Write a for loop to print the matches
# for match in matches:
#     print(match)

> item 1a.  risk factors       our business involves a high degree of risk. if any of the following risks, or any risk described elsewhere in this annual report on form 10-k, actually occur, our business, financial condition or results of operations could suffer. additional risks not presently known to us or which we currently consider immaterial also may adversely affect our company.  natural gas, ngl and oil prices are volatile. a substantial or extended decline in commodity prices may adversely affect our business, financial condition or results of operations and our ability to meet our capital expenditure obligations and financial commitments.      the prices we receive for our natural gas, ngl and oil production heavily influence our revenue, profitability, access to capital and future rate of growth. natural gas, ngls and oil are commodities and, therefore, their prices are subject to wide fluctuations in response to relatively minor changes in supply and demand. historically, th

In [ ]:
### Our goal is though to remove html tags and see the content
### Method get_text() is what we need, \n\n is optional, I just added this to read text 
### more cleanly, it's basically new line character between sections. 
print(item_7_content.get_text())

**Section 2: Numerical Data**